# Project Title
### Data Engineering Capstone Project

#### Project Summary
The primary objective of this project is to create a set of fact and dimensions tables and the supporting ETL pipeline to facilitate queries regarding visitors into the United States. The first step is to prepare staging tables in Redshift followed by setting up a set of fact and dimension tables based on the Star schema. Finally, an Airflow DAG is constructed to periodically run the process and check the data quality. 


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the mData
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [59]:
# Do all imports and installs here
import pandas as pd

pd.set_option('display.max_colwidth', 1)
pd.set_option('display.max_columns', 50)

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The I94_SAS_Labels_Descriptions.SAS provided in the data folder contains several dictionaries which are converted manually into the following .csv files:
* visa_type.csv - list of one-digit i94 visa type and their types
* i94mode.csv - list of one-digit i94 modes and their respective transportation mode
* i94port.csv - list of airport three-character codes, their names, and state/country
* i94cit.csv - list of three-digit country codes and their respective names
* state_code.csv - list of two-character state codes and their respective names



In [60]:
# Read in the data here
fname = 'data/immigration_data_sample.csv'
df = pd.read_csv(fname, index_col=0)

In [61]:
df.tail(10)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
2590789,5242730.0,2016.0,4.0,135.0,509.0,HAM,20572.0,1.0,MA,20575.0,47.0,2.0,1.0,20160428,HML,NaN,G,O,NaN,M,1969.0,10272016,F,NaN,DL,9.475131e+10,00560,B2
846327,1777652.0,2016.0,4.0,107.0,107.0,CHI,20554.0,1.0,IL,20565.0,51.0,1.0,1.0,20160410,WRW,NaN,G,O,NaN,M,1965.0,10092016,F,NaN,LH,9.319662e+10,00430,B1
1920712,3874218.0,2016.0,4.0,148.0,112.0,SFR,20565.0,1.0,CA,20582.0,49.0,2.0,1.0,20160421,NaN,NaN,O,O,NaN,M,1967.0,07192016,NaN,NaN,LH,5.653427e+10,00454,WT
451881,930868.0,2016.0,4.0,582.0,582.0,WAS,20549.0,1.0,DC,20552.0,42.0,2.0,1.0,20160405,GDL,NaN,G,O,NaN,M,1974.0,10042016,M,NaN,UA,9.282073e+10,01567,B2
2517187,5081809.0,2016.0,4.0,254.0,276.0,BOS,20571.0,1.0,MI,20582.0,51.0,2.0,1.0,20160427,NaN,NaN,G,O,NaN,M,1965.0,07252016,F,NaN,DL,5.932276e+10,00158,WT
2117909,4288772.0,2016.0,4.0,135.0,135.0,LVG,20567.0,1.0,NV,20572.0,32.0,2.0,1.0,20160423,NaN,NaN,G,O,NaN,M,1984.0,07212016,M,NaN,VS,5.914065e+10,00043,WT
1463022,2947585.0,2016.0,4.0,261.0,261.0,PSP,20560.0,1.0,HI,20567.0,35.0,1.0,1.0,20160416,JDD,NaN,G,O,NaN,M,1981.0,10152016,M,NaN,SV,9.371186e+10,00041,B1
1414569,2883298.0,2016.0,4.0,111.0,111.0,MIA,20560.0,1.0,FL,20566.0,39.0,2.0,1.0,20160416,NaN,NaN,G,O,NaN,M,1977.0,07142016,M,NaN,AF,5.627747e+10,00090,WT
1094181,2264857.0,2016.0,4.0,582.0,582.0,ATL,20556.0,1.0,WI,20559.0,35.0,1.0,1.0,20160412,MTR,NaN,G,O,NaN,M,1981.0,10112016,M,NaN,EV,9.334035e+10,05510,B1
2271807,4654865.0,2016.0,4.0,687.0,687.0,MIA,20568.0,1.0,FL,20578.0,44.0,2.0,1.0,20160424,BNS,NaN,G,O,NaN,M,1972.0,10232016,F,NaN,LA,9.440386e+10,02514,B2


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2027561 to 2271807
Data columns (total 28 columns):
cicid       1000 non-null float64
i94yr       1000 non-null float64
i94mon      1000 non-null float64
i94cit      1000 non-null float64
i94res      1000 non-null float64
i94port     1000 non-null object
arrdate     1000 non-null float64
i94mode     1000 non-null float64
i94addr     941 non-null object
depdate     951 non-null float64
i94bir      1000 non-null float64
i94visa     1000 non-null float64
count       1000 non-null float64
dtadfile    1000 non-null int64
visapost    382 non-null object
occup       4 non-null object
entdepa     1000 non-null object
entdepd     954 non-null object
entdepu     0 non-null float64
matflag     954 non-null object
biryear     1000 non-null float64
dtaddto     1000 non-null object
gender      859 non-null object
insnum      35 non-null float64
airline     967 non-null object
admnum      1000 non-null float64
fltno       992 non-null ob

In [63]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [64]:
#write to parquet
#df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

In [65]:
#df_spark.show(2)
df_spark.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,40.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1976.0,10292016,F,None,QF,9.495387e+10,00011,B1
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,32.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1984.0,10292016,F,None,VA,9.495562e+10,00007,B1
2,5748519.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,M,None,DL,9.495641e+10,00040,B1
3,5748520.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,F,None,DL,9.495645e+10,00040,B1
4,5748521.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,28.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1988.0,10292016,M,None,DL,9.495639e+10,00040,B1


In [66]:
# Data contains codes for airports all over the world
df_airport = pd.read_csv('data/airport-codes_csv.csv')

In [67]:
df_airport.tail()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
55070,ZYYK,medium_airport,Yingkou Lanqi Airport,0.0,AS,CN,CN-21,Yingkou,ZYYK,YKH,NaN,"122.3586, 40.542524"
55071,ZYYY,medium_airport,Shenyang Dongta Airport,NaN,AS,CN,CN-21,Shenyang,ZYYY,NaN,NaN,"123.49600219726562, 41.784400939941406"
55072,ZZ-0001,heliport,Sealand Helipad,40.0,EU,GB,GB-ENG,Sealand,NaN,NaN,NaN,"1.4825, 51.894444"
55073,ZZ-0002,small_airport,Glorioso Islands Airstrip,11.0,AF,TF,TF-U-A,Grande Glorieuse,NaN,NaN,NaN,"47.296388888900005, -11.584277777799999"
55074,ZZZZ,small_airport,Satsuma IÅjima Airport,338.0,AS,JP,JP-46,Mishima-Mura,RJX7,NaN,NaN,"130.270556, 30.784722"


In [68]:
df_airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [69]:
df_cities= pd.read_csv('data/us-cities-demographics.csv', delimiter =';')

In [70]:
df_cities.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [71]:
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [72]:
df_country_code = pd.read_csv('data/country_code.csv')
df_country_code.head(15)

,country_code,country_name
0,582,"MEXICO Air Sea, and Not Reported (I-94, no land arrivals)"
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA
5,324,ANGOLA
6,529,ANGUILLA
7,518,ANTIGUA-BARBUDA
8,687,ARGENTINA
9,151,ARMENIA


In [73]:
df_airport_code = pd.read_csv('data/i94port.csv')
df_airport_code.head()

,airport_code,airport_name,airport_state_or_country
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK


In [74]:
df_trans_mode = pd.read_csv('data/i94mode.csv')
df_trans_mode

,i94_mode,transportation_mode
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


In [75]:
df_state_code = pd.read_csv('data/state_code.csv')
df_state_code.head()

,state_code,state_name
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [76]:
df_visa = pd.read_csv('data/visa_type.csv')
df_visa

,i94_visa,visa_type
0,1,Business
1,2,Pleasure
2,3,Student


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

There are several columns with missing values in the immigration dataset. In particular, these columns have more than 5% of the rows missing and are listed in increasing percentage of missing values:
* depdate (5% missing)
* entdpd (5% missing)
* i94addr (6% missing)
* gender (15% missing)
* visapost (60% missing)
* insnum (96% missing)
* occup (99% missing)
* entdepu (100% missing)

The airport dataset has the following columns with missing values:
* iso_country (0.5% missing)
* municipality (10% missing)
* elevation_ft (13% missing)
* gps_code (26% missing)
* local_code (48% missing)
* continent (50% missing)
* iata_code (83% missing)

The columns with higher than 50% of its data missing are not used in creating the final fact and dimension tables. Imputation techniques such as filling in the average or most common value can only be useful when missing data is less than 15-20%.

The US cities demographics data contains very few null values in the following columns (Male Population, Female Population, Number of Veterans, Foreign-born, Average Household Size). As a result, this dataset does not need cleaning.

#### Cleaning Steps
The `i94port.csv` airport names and airport state/country columns have a lot of extra characters which had to be deleted. City names were sometimes characterized as country names which had to be corrected. Also leading and trailing spaces had to be trimmed. The `visa_type.csv`, `country_code.csv`, `state_code.csv` files required the quotation marks and equals signs removed to create the appropriate tables.  

The column `dtaddto` (date added to file) in the immigration dataset had null values as 'D/S'. To accommodate this, when creating the staging_immigration table in Redshift, the column is assigned a VARCHAR data type.  

### Rolling up cities data into states

The `cities` dataset contains information on individual cities but since the immigration dataset only reports the state (not city) of arrival, the cities data is rolled up into states. The median age and household size columns are aggregated using the `mean` function while the population descriptions are rolled up using `sum`. 


In [77]:
# Roll up city data into states and convert some columns into fractions of total population

df_states = df_cities.groupby("State Code").agg({"Median Age": "mean",
                                     "Male Population": "sum",
                                     "Female Population": "sum",
                                     "Total Population": "sum",
                                     "Number of Veterans": "sum",
                                     "Foreign-born": "sum",
                                     "Average Household Size": "mean",
                                     "Count": "count"})

# Convert summed values into fraction of total population
df_states["Male Population"] = df_states["Male Population"]/df_states["Total Population"]
df_states["Female Population"] = df_states["Female Population"]/df_states["Total Population"]
df_states["Number of Veterans"] = df_states["Number of Veterans"]/df_states["Total Population"]
df_states["Foreign-born"] = df_states["Foreign-born"]/df_states["Total Population"]

In [78]:
# Join df_states with df_state_code
df_joined_states = df_state_code.join(df_states, how='left', on='state_code')

In [79]:
# Create a new csv file with states data
df_joined_states.to_csv('data/us_states.csv')

In [80]:
df_joined_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 10 columns):
state_code                55 non-null object
state_name                55 non-null object
Median Age                49 non-null float64
Male Population           49 non-null float64
Female Population         49 non-null float64
Total Population          49 non-null float64
Number of Veterans        49 non-null float64
Foreign-born              49 non-null float64
Average Household Size    48 non-null float64
Count                     49 non-null float64
dtypes: float64(8), object(2)
memory usage: 4.4+ KB


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
To accommodate queries on immigration arrival data, the following facts and dimension tables are created following the Star schema model. The central fact table contains principal information on I-94 form and means of arrival into the United States. The dimensions table provide more detailed information on states, airports, countries, time, and admissions.  

![Facts and dimension tables](images/fact_dimension_tables.png)

Using the schema above, the following queries can be easily performed:
* Airports with the most visitors
* Age distribution of visitors into the United States
* States with the most number of business visitors per population each year
* States with the most number of residents born in a foreign country 

#### 3.2 Mapping Out Data Pipelines
To pipeline the data into the chosen model, the first step is to stage the necessary data into staging tables in Redshift:

![Staging tables](images/staging_tables.png)

Then the data is extracted from these preliminary tables into the final fact and dimensions tables.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
The following SQL commands are used to create staging tables as well as the final fact and dimension tables in Redshift.

In [81]:
####################################################
## SQL Queries to build tables in Redshift#########

create_tables= ("""
CREATE TABLE IF NOT EXISTS public.staging_immigration (
     cicid FLOAT,
     i94yr FLOAT,
     i94mon FLOAT,
     i94cit FLOAT,
     i94res FLOAT,
     i94port VARCHAR,
     arrdate FLOAT,
     i94mode FLOAT,
     i94addr VARCHAR,
     depdate FLOAT,
     i94bir FLOAT,
     i94visa FLOAT,
     count FLOAT,
     dtadfile VARCHAR,
     visapost VARCHAR,
     occup VARCHAR,
     entdepa VARCHAR,
     entdepd VARCHAR,
     entdepu VARCHAR,
     matflag VARCHAR,
     biryear FLOAT,
     dtaddto VARCHAR,
     gender VARCHAR,
     insnum VARCHAR,
     airline VARCHAR,
     admnum FLOAT,
     fltno VARCHAR,
     visatype VARCHAR
);
     
CREATE TABLE IF NOT EXISTS public.states (
     state_code VARCHAR,
     state_name VARCHAR,
     median_age FLOAT,
     male_population FLOAT,
     female_population FLOAT,
     total_population INTEGER,
     number_of_veterans FLOAT,
     foreign_born FLOAT,
     household_size FLOAT,
     city_count INTEGER,
     CONSTRAINT states_pkey PRIMARY KEY (state_code)
);
     
CREATE TABLE IF NOT EXISTS public.airport_code (
     airport_code VARCHAR,
     airport_name VARCHAR,
     airport_state VARCHAR,
     CONSTRAINT airport_pkey PRIMARY KEY (airport_code)
);    
     
CREATE TABLE IF NOT EXISTS public.countries (
     country_code VARCHAR,
     country_name VARCHAR,
     CONSTRAINT countries_pkey PRIMARY KEY (country_code)
); 

CREATE TABLE IF NOT EXISTS public.staging_trans_mode (
     i94_mode INTEGER,
     transportation_mode VARCHAR,
     CONSTRAINT trans_mode_pkey PRIMARY KEY (i94_mode)
); 
     
CREATE TABLE IF NOT EXISTS public.staging_visa_code (
     i94_visa INTEGER,
     visa_type VARCHAR,
     CONSTRAINT visa_pkey PRIMARY KEY (i94_visa)
); 
     
CREATE TABLE IF NOT EXISTS public.arrivals (
     cicid INTEGER,
     state_code INTEGER,
     country_code INTEGER,
     airport_code VARCHAR,
     arrdate INTEGER,
     arrival_code VARCHAR,
     arrival_mode VARCHAR,
     state VARCHAR,
     airline VARCHAR,
     admnum BIGINT,
     fltno VARCHAR
);
     
CREATE TABLE IF NOT EXISTS public.admissions (
     admnum BIGINT,
     gender VARCHAR,
     age VARCHAR,
     biryear TIMESTAMP,
     visa_code INTEGER,
     visa_type VARCHAR,
     CONSTRAINT admissions_pkey PRIMARY KEY (admnum)
);

CREATE TABLE IF NOT EXISTS public.time (
    arrdate TIMESTAMP,
    month int4,
    year int4,
    week int4,
    day int4,
    weekday int4,
    CONSTRAINT time_pkey PRIMARY KEY (arrdate)
);
""")
        

Once staging and final tables are created in Redshift, all the data is then copied to the staging tables using the following SQL commands. The US immigration data comes in Parquet format while the airport and US states data are in CSV format thus different commands are used for each data type:

In [82]:
####################################################
## SQL Queries to copy csv tables into Redshift#####

copy_sql = """
COPY {}
FROM '{}'
ACCESS_KEY_ID '{}'
SECRET_ACCESS_KEY '{}'
IGNOREHEADER 1
CSV;
"""

########################################################
## SQL Queries to copy parquet tables into Redshift#####

copy_parquet_sql = """
COPY {}
FROM '{}'
IAM_ROLE '{}'
FORMAT AS PARQUET;
"""

Then, data is transferred from the staging tables into the fact and dimension tables. The following SQL commands can be used:

In [83]:
##################################################
## SQL Queries to insert tables into Redshift#####
        
arrivals_table_insert = ("""
SELECT CAST(im.cicid AS INT),
       CAST(im.i94cit AS INT) AS state_code,
       CAST(im.i94res AS INT) AS country_code,
       im.i94port AS airport_code,
       CAST(arrdate AS INT),
       CAST(im.i94mode AS INT) AS arrival_code,
       tm.transportation_mode AS arrival_mode,
       im.i94addr AS state,
       im.airline,
       CAST(im.admnum AS BIGINT),
       im.fltno
    FROM staging_immigration AS im
    LEFT JOIN staging_trans_mode AS tm
    ON CAST(im.i94mode AS INT) = tm.i94_mode
""")

admissions_table_insert = ("""
SELECT CAST(im.admnum AS BIGINT),
       im.gender,
       CAST(im.i94bir AS INT),
       to_timestamp(CAST(im.biryear AS INT), 'YYYY') as birth_year,
       CAST(im.i94visa AS INT),
       vs.visa_type
    FROM staging_immigration AS im
    LEFT JOIN staging_visa_code AS vs
    ON CAST(im.i94visa AS INT) = vs.visa_type
""")

time_table_insert = ("""
SELECT dates.arrtime AS arrdate,
       extract(year from dates.arrtime),
       extract(month from dates.arrtime),
       extract(week from dates.arrtime), 
       extract(day from dates.arrtime),
       extract(dayofweek from dates.arrtime) AS weekday
    FROM (SELECT DATEADD(day, CAST(arrdate AS INT), '1900-01-01') AS arrtime
        FROM staging_immigration) dates
""")

The overall ETL process is then built into a pipeline in Airflow with a daily update schedule. Data quality checks are added at the end of the pipeline to ensure no missing values for the primary keys.
![Airflow graph view DAG](images/dag_image.png)

5 custom operators are created in Airflow to . Airflow Connections are used to store two credentials: the first credential is a set of AWS access key ID and secret access key for retrieving the raw data on S3; the second credential is a set of database name, username, password, endpoint, and port information to connect to the Redshift database.

There is a `aws.cfg` file in the "dags" folder which contains the Amazon Resource Name (ARN) of the user's AWS Identity and Access Management (IAM) role which would allow access to the Redshift database.

#### 4.2 Data Quality Checks
Data quality checks to run:
 * Integrity check on primary keys of fact and dimensional tables
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [84]:
# Data quality checks to run in SQL
data_quality_checks =[
 {'check_sql': "SELECT COUNT(*) FROM arrivals WHERE cic_id IS NULL", 'expected_result':0},   
 {'check_sql': "SELECT COUNT(*) FROM admissions WHERE admnum IS NULL", 'expected_result':0},
 {'check_sql': "SELECT COUNT(*) FROM time WHERE arrdate IS NULL", 'expected_result':0},
 {'check_sql': "SELECT COUNT(*) FROM states WHERE state_code IS NULL", 'expected_result':0},
 {'check_sql': "SELECT COUNT(*) FROM states WHERE state_code IS NULL", 'expected_result':0}
]

Upon completion of the steps in the pipeline, the following tree view can be seen on the AirFlow UI:
![Airflow Success DAG Tree View](images/success_dag.png)


#### 4.3 Data dictionary 
Data dictionary for the tables:

#### arrival table:
* cicid: US immigration identification number
* state_code: 2-character state code
* country_code: 3-digit country code
* airport_code: 3-character airport code
* arrdate: arrival date (raw: number of days since 1/1/1900 SAS type number)
* arrival_code: 1-digit arrival mode
* arrival_mode: mode of transportation to arrive into the US (air, sea, or land)
* state: 2-character state code
* airline: airline used to enter the US
* admnum: admission number
* fltno: flight number and airlines used

#### states table:
* state_code: 2-character state code 
* state_name: state name
* median_age: median age of people in the state
* male_population: fraction of male population in the state
* female_population: fraction of female population in the state
* total_population: number of population based on cities in the database
* number_veterans: fraction of veterans in the population
* foreign_born: fraction of people that are born in a foreign country
* household_size: average number of people in a household
* city_count: number of cities accounted for in the state

#### admissions table:
* adm_num: admission number
* gender: gender of person arriving in the US
* age: age of person arriving
* biryear: year of birth of person arriving
* visa_code: 1 digit code visa type
* visa_type: type of visa (e.g. business, student, pleasure)

#### time table:
* arrdate: arrival date
* year: year of arrival
* month: month of arrival
* week: week of the year of arrival (e.g. 1-52)
* day: day of the year of arrival (e.g. 1-365)
* weekday: weekday of arrival (e.g. 1-7)

#### airports table:
* airport_code: 3-character airport code
* airport_name: name of airport
* airport_state: state that airport is located in

#### countries table:
* country_code: 3-digit country code
* country_name: name of country

#### Justification for technology stack
* AWS S3 is chosen as the storage platform due to its low cost, high availability, and compatibility with AWS Redshift.

* Redshift is selected to host the relational table since it allows for easy start-up process, security, and fast scaling. Further, it has an SQL interface and works very well with data stored in AWS S3 (high performance parallel loading).

* Airflow is selected as the primary technology to schedule and track the workflow in the ETL pipeline. Airflow provides a simple user interface and the ability to monitor tasks and send alerts in case any of the process fails. In addition, Airflow is easily extendable through the use of custom operators, hooks, and sensors. 

#### Additional thoughts

* The database can be updated on a daily basis as there are over 80 million visitors to the United States every year which is roughly 7 million visitors per month or 230,000 visitors per week. This schedule allows for frequent updates while not requiring too much resources (compute and duration) on each update.  


* In the scenario that the data was increased by 100x:
    * One alternative would be to host the data on multiple clusters of machine and use Spark SQL to leverage the power of distributed computing in order to speed up the data ingestion and processing steps. 
    
    
* In the scenario that the data populates a dashboard that must be updated on a daily basis by 7am every day:
    * I would continue to use AirFlow to update the data but will add a service level agreement to ensure that the data processing is completed before the daily deadline. An alert email can be set up to indicate failure along the process to allow early detection.


* In the scenario that the database needed to be accessed by 100+ people:
    * I would chooose a database system that is compatible with Spark as Spark has powerful built-in tools to handle concurrent operations by scheduling work in a sequential manner while still enabling parallelism.